In [ ]:
import matplotlib.pyplot as plt # pour graphiques

In [ ]:
# Mise en forme des données

import pandas as pd
import numpy as np


def file_to_dataframe(filenames,st): 
    """
    Initialize the DataFrame from a filenames 
    Parameters: 
    -----------
    filenames: str, the name of the file
    st: str, helps to delimites the columns of the datas because "Correspondances_promo
    are delimited with ';' while the other are delimited with ','.
    """
    datas = pd.read_csv(filenames,delimiter=st)
    return datas


def clean_dates(df):
    """
    Clean the dates of the files with dates such as "2022-11-14T00:00:00Z" and change it into "2022-11-14"
    -----------
    df: dataframe, the dataframe that we use
    
    """
    for colu in df.columns :
        if df[colu].dtype == object: 
            df[colu] = df[colu].str.replace('T00:00:00Z', '')
    return(df)

def change_date(st):
    """
    Change a date as "28/04/2003 00:00:00.000" into one more precise "2003-04-28"
    -----------
    st: str, the date to change

    """ 
    if type(st) != str or len(st) < 10 : 
        return(st)
    else :
        date = st[:10].split("/")
        return(date[2]+"-"+date[1]+"-"+date[0])
    
def change_dates_all(df,cols):
    """
    Change the dates of some columns of a dataFrame using the function change_date
    -----------
    df: dataframe, the DataFrame on which we are working
    cols: str list, a list of columns name that we want to change

    """ 
    for colu in cols : 
        df[colu] = df[colu].apply(change_date)
    return df

def save_to_csv_file(df,filename):
    """
    Save a dataframe on a .csv file at filename
    ----------
    df: dataframe, the DataFrame that you want to save
    filename: str, the place where you want to save your data frame
    """
    df.to_csv(filename, index=False)
    return True

In [ ]:
df = file_to_dataframe("/Users/antoine/Documents/ENSAE 2A/Code perso/everything/Statapp perso/Ressources/Correspondances_Promos.csv",";")
df_Correspondances_Promos = change_dates_all(df,['DEBVAL', 'FINVAL', 'DEBABOMIN', 'DEBABOMAX'])
df_Correspondances_Promos.head()

In [ ]:
df = file_to_dataframe("/Users/antoine/Documents/ENSAE 2A/Code perso/everything/Statapp perso/Ressources/Données_Promos_2021.csv",",")
df_Données_Promo_2021 = clean_dates(df)
df_Données_Promo_2021.head()

In [ ]:
df = file_to_dataframe("/Users/antoine/Documents/ENSAE 2A/Code perso/everything/Statapp perso/Ressources/Données_Reabos_2021.csv",",")
df_Données_Reabos_2021 = clean_dates(df)
df_Données_Reabos_2021.head()

In [ ]:
def join_dataFrames(df1,df2,cond):
    """
    Join two DataFrames on the conditions cond
    Parameters: 
    -----------
    df1,df2: DataFrames
    cond: str list of the colomuns on which we want to join 
    """
    return pd.merge(df1, df2, on=cond, how='inner')


In [ ]:
def calculate_nb_promos(df):
    return(df.groupby('CPROMO')['ID_ABONNE'].count())

In [ ]:
promo_reabo = join_dataFrames(df_Données_Promo_2021,df_Données_Reabos_2021,['ID_ABONNE','DATE_ACTE_REEL'])
promo_reabo.head()

In [ ]:
df_nb_promos = calculate_nb_promos(promo_reabo)
print((df_nb_promos).sort_values(ascending=False))

In [ ]:
print(df_Données_Promo_2021.shape)
print(df_Données_Reabos_2021.shape)
print(promo_reabo.shape)

In [ ]:
# Next : Objectif : comprendre les comportements de réabonnement des abonnés 
# en fonction des différents types de promotions (Semaine généreuse vs autres ODD vs hors promotion)

In [ ]:
# 1) Identifier différents types d'ODD :
# => semaine généreuse (EV+/TC) => SG
# => ODD 7 jours autre que SG
# => ODD 15 jours EV+
# => ODD 15 jours TC
# => ODD 21 jours EV+
# => ODD 21 jours TC
# => Autres types d'ODD ?

In [ ]:
df_join = join_dataFrames(promo_reabo, df_Correspondances_Promos, ['CPROMO']) # Join ultime


In [ ]:
df_join.head()
df_join.shape

In [ ]:
odd = df_join[df_join['TYPE_PROMO'] == 'ODD'] # Que les ODD
nodd = df_join[df_join['TYPE_PROMO'] != 'ODD'] # Que les non ODD

In [ ]:
q1 = odd.groupby(['CPROMO', 'AVANTAGE'])['ID_ABONNE'].count() # Code promo les + utilisés
q1 = q1.sort_values(ascending=False)
q1

In [ ]:
q1b = odd.groupby(['AVANTAGE'])['ID_ABONNE'].count() # Avantages les + utilisés
q1b = q1b.sort_values(ascending=False)
q1b

In [ ]:
semgen = odd.groupby(['AVANTAGE', 'SG'])['ID_ABONNE'].count() # Par avantage 
semgen = semgen.sort_values(ascending=False)
semgen

In [ ]:
oddnsg = odd[odd['SG'].astype(str) != 'Semaine généreuse'] # ODD non SG

In [ ]:
analyse = oddnsg.groupby(['AVANTAGE'])['ID_ABONNE'].count() # ODD non SG classées par utilisation
analyse = analyse.sort_values(ascending=False)
analyse 

In [ ]:
odd7jnsg = oddnsg[oddnsg['AVANTAGE'].isin(['EVPDD 7 jour(s)', 'EVDD 7 jour(s)'])] # ODD de 7j sans semaine généreuse
odd7jnsg

In [ ]:
# 2) Répartition des réabonnements par type de promotions et hors promotion par année/mois :
# => total pays
# => par region / secteur
# => canal de distribution / enseigne
# => moyens de paiement
# => formule prec

In [ ]:
# 3) Répartition des réabonnements par délai de réabonnement (depuis fin de l'abonnement précédent) 
#    par type de promotions et hors promotion.

In [ ]:
# Création d'une variable qui calcule le délai de réabonnement 

# On met les dates au bon format 
df_Données_Reabos_2021['DATE_ACTE_REEL'] = pd.to_datetime(df_Données_Reabos_2021['DATE_ACTE_REEL'])
df_Données_Reabos_2021['DATE_FIN_ABO_PREC'] = pd.to_datetime(df_Données_Reabos_2021['DATE_FIN_ABO_PREC'])

In [47]:
df_Données_Reabos_2021['delai_reabo'] = (df_Données_Reabos_2021['DATE_ACTE_REEL'] - df_Données_Reabos_2021['DATE_FIN_ABO_PREC']).dt.days

# Delai_reabo < 0 : le client a pris son nouvel abonnement avant expiration 

In [48]:
df_Données_Promo_2021['DATE_ACTE_REEL'] = pd.to_datetime(df_Données_Promo_2021['DATE_ACTE_REEL']) # Obligé pour fusionner

In [ ]:
promo_reabo2 = join_dataFrames(df_Données_Promo_2021,df_Données_Reabos_2021,['ID_ABONNE','DATE_ACTE_REEL'])
promo_reabo2.head() # Nouvelle table avec la variable du délai de réabonnement

In [56]:
df_join2 = join_dataFrames(promo_reabo2, df_Correspondances_Promos, ['CPROMO'])

In [57]:
# On s'interesse dans un premier temps avec promo
moyenne_par_CPROMO = df_join2.groupby(['CPROMO','AVANTAGE'])['delai_reabo'].mean()
moyenne_par_CPROMO

CPROMO    AVANTAGE         
160412B2  EVPDD 7 jour(s)      33.978723
160412B3  EVPDD 7 jour(s)      32.533568
160412B6  EVPDD 7 jour(s)      20.283854
160412C2  TCAODD 7 jour(s)     38.500000
160412C3  TCAODD 7 jour(s)     21.050734
                                 ...    
SGTCA01   TCAODD 7 jour(s)     -3.071011
SGTCA03   TCAODD 21 jour(s)   -20.150659
SGTCA06   TCAODD 42 jour(s)   -26.842105
SGTCA12   TCAODD 84 jour(s)   -19.438095
SGTCAAN   TCAODD 1 jour(s)     33.800000
Name: delai_reabo, Length: 133, dtype: float64

In [63]:
# Puis pour ceux sans promo 
# j'arrive pas a anti join les deux tableaux reabo et promo_reabo pour n'avoir plus que ceux qui n'ont pas 
# bénéficié d'une promo en se réabonnant

In [ ]:
# 4) Répartition des réabonnements par taux de consommation (12 mois glissants) 
#    par type de promotions et hors promotion.

In [ ]:
# C'est quoi taux de consommation ?